# Save and load models 

Model progress can be saved during—and after—training. This means a model can resume where it left off and avoid long training times. Saving also means you can share your model and others can recreate your work. When publishing research models and techniques, most machine learning practitioners share:

* code to create the model, and
* the trained weights, or parameters, for the model

Sharing this data helps others understand how the model works and try it themselves with new data.

Caution: Be careful with untrusted code—TensorFlow models are code. See [Using TensorFlow Securely](https://github.com/tensorflow/tensorflow/blob/master/SECURITY.md) for details.

### Options

There are different ways to save TensorFlow models—depending on the API you're using. This guide uses [tf.keras](https://www.tensorflow.org/guide/keras), a high-level API to build and train models in TensorFlow. For other approaches, see the TensorFlow  [Save and Restore](https://www.tensorflow.org/guide/saved_model) guide or [Saving in eager](https://www.tensorflow.org/guide/eager#object-based_saving).

## Setup

### Installs and imports

##### Install and import TensorFlow and dependencies:

In [1]:
!pip install -q pyyaml h5py  # Required to save models in HDF5 format

In [2]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.0.0


### Get an example dataset

To demonstrate how to save and load weights, you'll use the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). To speed up these runs, use the first 1000 examples:

In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

### Define a model

Start by building a simple sequential model:

In [4]:
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512,activation='relu',input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10,activation='softmax')
    ])
    model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
    return model

In [5]:
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


## Save checkpoints during training

You can use a trained model without having to retrain it, or pick-up training where you left off—in case the training process was interrupted. The `tf.keras.callbacks.ModelCheckpoint` callback allows to continually save the model both *during* and at *the end* of training.

### Checkpoint callback usage

Create a `tf.keras.callbacks.ModelCheckpoint` callback that saves weights only during training:

In [6]:
checkpoint_path = 'training_1/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

# create a callback that saves model weight
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                save_weights_only=True,verbose=1)
#train the model with the new callback
model.fit(train_images,
          train_labels,
         epochs=10,
         validation_data=(test_images,test_labels),
          callbacks = [cp_callback]
         )

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 768/1000 [======================>.......] - ETA: 0s - loss: 1.3078 - accuracy: 0.6237
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 1s 674us/sample - loss: 1.1667 - accuracy: 0.6700 - val_loss: 0.7177 - val_accuracy: 0.7830
Epoch 2/10
 640/1000 [==================>...........] - ETA: 0s - loss: 0.4711 - accuracy: 0.8672
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 214us/sample - loss: 0.4275 - accuracy: 0.8820 - val_loss: 0.5264 - val_accuracy: 0.8450
Epoch 3/10
 672/1000 [===================>..........] - ETA: 0s - loss: 0.2743 - accuracy: 0.9315
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 207us/sample - loss: 0.2962 - accuracy: 0.9220 - val_loss: 0.5008 - val_accuracy: 0.8420
Epoch 4/10
 832/1000 [=======================>......] - ETA: 0s - loss: 0.2122 - accuracy: 0.9507
Epoch

This creates a single collection of TensorFlow checkpoint files that are updated at the end of each epoch:

In [7]:
!ls {checkpoint_dir}

checkpoint  cp.ckpt.data-00000-of-00001  cp.ckpt.index


Create a new, untrained model. When restoring a model from weights-only, you must have a model with the same architecture as the original model. Since it's the same model architecture, you can share weights despite that it's a different *instance* of the model.

Now rebuild a fresh, untrained model, and evaluate it on the test set. An untrained model will perform at chance levels (~10% accuracy):

In [8]:
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

1000/1 - 0s - loss: 2.3298 - accuracy: 0.1000
Untrained model, accuracy: 10.00%


Then load the weights from the checkpoint and re-evaluate:

In [9]:
# Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1 - 0s - loss: 0.4123 - accuracy: 0.8630
Restored model, accuracy: 86.30%


### Checkpoint callback options

The callback provides several options to provide unique names for checkpoints and adjust the checkpointing frequency.

Train a new model, and save uniquely named checkpoints once every five epochs:

In [10]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    periods = 5
)

# Create a new model instance
model = create_model()

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model.fit(train_images, 
              train_labels,
              epochs=50, 
              callbacks=[cp_callback],
              validation_data=(test_images,test_labels),
              verbose=0)


Epoch 00001: saving model to training_2/cp-0001.ckpt

Epoch 00002: saving model to training_2/cp-0002.ckpt

Epoch 00003: saving model to training_2/cp-0003.ckpt

Epoch 00004: saving model to training_2/cp-0004.ckpt

Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00006: saving model to training_2/cp-0006.ckpt

Epoch 00007: saving model to training_2/cp-0007.ckpt

Epoch 00008: saving model to training_2/cp-0008.ckpt

Epoch 00009: saving model to training_2/cp-0009.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00011: saving model to training_2/cp-0011.ckpt

Epoch 00012: saving model to training_2/cp-0012.ckpt

Epoch 00013: saving model to training_2/cp-0013.ckpt

Epoch 00014: saving model to training_2/cp-0014.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00016: saving model to training_2/cp-0016.ckpt

Epoch 00017: saving model to training_2/cp-0017.ckpt

Epoch 00018: saving model to training_2/cp-0018.ckpt

Epoch 00019: saving model t

Now, look at the resulting checkpoints and choose the latest one:

In [11]:
!ls {checkpoint_dir}

checkpoint			  cp-0025.ckpt.index
cp-0000.ckpt.data-00000-of-00001  cp-0026.ckpt.data-00000-of-00001
cp-0000.ckpt.index		  cp-0026.ckpt.index
cp-0001.ckpt.data-00000-of-00001  cp-0027.ckpt.data-00000-of-00001
cp-0001.ckpt.index		  cp-0027.ckpt.index
cp-0002.ckpt.data-00000-of-00001  cp-0028.ckpt.data-00000-of-00001
cp-0002.ckpt.index		  cp-0028.ckpt.index
cp-0003.ckpt.data-00000-of-00001  cp-0029.ckpt.data-00000-of-00001
cp-0003.ckpt.index		  cp-0029.ckpt.index
cp-0004.ckpt.data-00000-of-00001  cp-0030.ckpt.data-00000-of-00001
cp-0004.ckpt.index		  cp-0030.ckpt.index
cp-0005.ckpt.data-00000-of-00001  cp-0031.ckpt.data-00000-of-00001
cp-0005.ckpt.index		  cp-0031.ckpt.index
cp-0006.ckpt.data-00000-of-00001  cp-0032.ckpt.data-00000-of-00001
cp-0006.ckpt.index		  cp-0032.ckpt.index
cp-0007.ckpt.data-00000-of-00001  cp-0033.ckpt.data-00000-of-00001
cp-0007.ckpt.index		  cp-0033.ckpt.index
cp-0008.ckpt.data-00000-of-00001  cp-0034.ckpt.data-00000-of-00001
cp-0008.ckpt.index		  cp-0034.ckpt.

In [12]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

Note: the default tensorflow format only saves the 5 most recent checkpoints.

To test, reset the model and load the latest checkpoint:

In [13]:
# Create a new model instance
model = create_model()

# Load the previously saved weights
model.load_weights(latest)

# Re-evaluate the model
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1 - 0s - loss: 0.5579 - accuracy: 0.8800
Restored model, accuracy: 88.00%


## What are these files?

The above code stores the weights to a collection of [checkpoint](https://www.tensorflow.org/guide/saved_model#save_and_restore_variables)-formatted files that contain only the trained weights in a binary format. Checkpoints contain:
* One or more shards that contain your model's weights.
* An index file that indicates which weights are stored in a which shard.

If you are only training a model on a single machine, you'll have one shard with the suffix: `.data-00000-of-00001`

## Manually save weights

You saw how to load the weights into a model. Manually saving them is just as simple with the `Model.save_weights` method. By default, `tf.keras`—and `save_weights` in particular—uses the TensorFlow [checkpoints](../../guide/keras/checkpoints) format with a `.ckpt` extension (saving in [HDF5](https://js.tensorflow.org/tutorials/import-keras.html) with a `.h5` extension is covered in the [Save and serialize models](../../guide/keras/save_and_serialize#weights-only_saving_in_savedmodel_format) guide):

In [14]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Create a new model instance
model = create_model()

# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1 - 0s - loss: 0.5579 - accuracy: 0.8800
Restored model, accuracy: 88.00%


## Save the entire model

Call [`model.save`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#save) to save the a model's architecture, weights, and training configuration in a single file/folder. This allows you to export a model so it can be used without access to the original Python code*. Since the optimizer-state is recovered, you can resume training from exactly where you left off.

Saving a fully-functional model is very useful—you can load them in TensorFlow.js ([HDF5](https://js.tensorflow.org/tutorials/import-keras.html), [Saved Model](https://js.tensorflow.org/tutorials/import-saved-model.html)) and then train and run them in web browsers, or convert them to run on mobile devices using TensorFlow Lite ([HDF5](https://www.tensorflow.org/lite/convert/python_api#exporting_a_tfkeras_file_), [Saved Model](https://www.tensorflow.org/lite/convert/python_api#exporting_a_savedmodel_))

\*Custom objects (e.g. subclassed models or layers) require special attention when saving and loading. See the **Saving custom objects** section below 

### HDF5 format

Keras provides a basic save format using the [HDF5](https://en.wikipedia.org/wiki/Hierarchical_Data_Format) standard. 

In [15]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model shuold be saved to HDF5.
model.save('my_model.h5') 

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 336us/sample - loss: 1.1152 - accuracy: 0.6840
Epoch 2/5
1000/1000 [==============================] - 0s 124us/sample - loss: 0.4251 - accuracy: 0.8790
Epoch 3/5
1000/1000 [==============================] - 0s 117us/sample - loss: 0.2813 - accuracy: 0.9260
Epoch 4/5
1000/1000 [==============================] - 0s 114us/sample - loss: 0.2215 - accuracy: 0.9330
Epoch 5/5
1000/1000 [==============================] - 0s 119us/sample - loss: 0.1512 - accuracy: 0.9620


In [16]:
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('my_model.h5')

# Show the model architecture
new_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [17]:
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

1000/1 - 0s - loss: 0.4896 - accuracy: 0.8620
Restored model, accuracy: 86.20%


This technique saves everything:

* The weight values
* The model's configuration(architecture)
* The optimizer configuration

Keras saves models by inspecting the architecture. Currently, it is not able to save TensorFlow optimizers (from `tf.train`). When using those you will need to re-compile the model after loading, and you will lose the state of the optimizer.


### SavedModel format

The SavedModel format is another way to serialize models. Models saved in this format can be restored using `tf.keras.models.load_model` and are compatible with TensorFlow Serving. The [SavedModel guide](https://www.tensorflow.org/guide/saved_model) goes into detail about how to serve/inspect the SavedModel. The section below illustrates the steps to saving and restoring the model.

In [18]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model') 

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 340us/sample - loss: 1.1752 - accuracy: 0.6600
Epoch 2/5
1000/1000 [==============================] - 0s 120us/sample - loss: 0.4249 - accuracy: 0.8890
Epoch 3/5
1000/1000 [==============================] - 0s 119us/sample - loss: 0.3021 - accuracy: 0.9200
Epoch 4/5
1000/1000 [==============================] - 0s 116us/sample - loss: 0.2103 - accuracy: 0.9500
Epoch 5/5
1000/1000 [==============================] - 0s 116us/sample - loss: 0.1542 - accuracy: 0.9740
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [19]:
# my_model directory
!ls saved_model

# Contains an assets folder, saved_model.pb, and variables folder.
!ls saved_model/my_model

my_model
assets	saved_model.pb	variables


In [20]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [21]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

print(new_model.predict(test_images).shape)

1000/1 - 0s - loss: 0.5873 - accuracy: 0.8580
Restored model, accuracy: 85.80%
(1000, 10)


### Saving custom objects

If you are using the SavedModel format, you can skip this section. The key difference between HDF5 and SavedModel is that HDF5 uses object configs to save the model architecture, while SavedModel saves the execution graph. Thus, SavedModels are able to save custom objects like subclassed models and custom layers without requiring the orginal code.

To save custom objects to HDF5, you must do the following:

1. Define a `get_config` method in your object, and optionally a `from_config` classmethod.
  * `get_config(self)` returns a JSON-serializable dictionary of parameters needed to recreate the object.
  * `from_config(cls, config)` uses the returned config from `get_config` to create a new object. By default, this function will use the config as initialization kwargs (`return cls(**config)`).
2. Pass the object to the `custom_objects` argument when loading the model. The argument must be a dictionary mapping the string class name to the Python class. E.g. `tf.keras.models.load_model(path, custom_objects={'CustomLayer': CustomLayer})`

See the [Writing layers and models from scratch](https://www.tensorflow.org/guide/keras/custom_layers_and_models) tutorial for examples of custom objects and `get_config`.
